<br>  

## <span style='color:blue'>Section 1: Import</span>  

In [1]:
import pandas as pd
import numpy as np
import copy

from sklearn.model_selection import StratifiedKFold

from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from collections import Counter

from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

<br>  

## <span style='color:blue'>Section 2: Read, drop columns, form X_train and y_train</span>  

In [2]:
# ---------- read ----------

df_1 = pd.read_csv('../data/code_3_train.csv')

# ---------- drop ----------

print('Before drop :', df_1.shape)
df_1.drop(columns=['index',
                   'pco2', 'ph', 'basophils', 'lactic_acid', 'bmi',
                   'creatine_kinase', 'lymphocyte', 'neutrophils'], inplace=True)
print('After drop:', df_1.shape)
print('')

# ---------- form X ----------

X_train = df_1.drop(columns=['outcome'])
print('X_train :', X_train.shape)

# ---------- form y ----------

y_train = df_1['outcome']
print('y_train :', y_train.shape)
print('')
print('y_train :', np.unique(y_train, return_counts=True))
print('y_train :', Counter(y_train))
print(y_train.value_counts(normalize=True))

Before drop : (882, 51)
After drop: (882, 42)

X_train : (882, 41)
y_train : (882,)

y_train : (array([0., 1.]), array([763, 119], dtype=int64))
y_train : Counter({0.0: 763, 1.0: 119})
0.0    0.865079
1.0    0.134921
Name: outcome, dtype: float64


<br>  

## <span style='color:blue'>Section 3: Set up 15-fold cross validation</span>  

In [3]:
kfold_cv = StratifiedKFold(n_splits=15, shuffle=True, random_state=42)

<br>  

## <span style='color:blue'>Section 4: Set up neural network model in function</span>  

In [4]:
def nn_keras_class(layer_1, layer_2, dropout_rate, regularisation_alpha):
    model = Sequential()
    model.add(Dense(units=layer_1, activation='relu', input_dim=41))
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(units=layer_2, activation='relu', kernel_regularizer=l1(regularisation_alpha)))
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['Recall'])
    return model

nn_keras_class_model = KerasClassifier(build_fn=nn_keras_class, epochs=100, batch_size=32, verbose=2)

<br>  

## <span style='color:blue'>Section 5: Set up neural network model, pipeline, hyperparameter search, early stopping</span>  

In [5]:
nn_keras_pipeline = Pipeline([
    ('knn_impute', KNNImputer(n_neighbors=5)),
    ('smote_sample', SMOTE(random_state=42)),
    ('std_scale', StandardScaler()),
    ('nn_model', nn_keras_class_model)
])

nn_keras_param_grid = {
    'nn_model__layer_1': [5, 6],
    'nn_model__layer_2': [3, 4],
    'nn_model__dropout_rate': [0.0, 0.2, 0.4],
    'nn_model__regularisation_alpha': [0.1, 0.01, 0.001, 0.0001, 0.00001]
}

es = EarlyStopping(patience=3, monitor='loss')

<br>  

## <span style='color:blue'>Section 6: GridSearchCV instantiate and fit </span>  

In [6]:
nn_keras_gridsearchcv = GridSearchCV(nn_keras_pipeline,
                                     param_grid=nn_keras_param_grid,
                                     scoring='recall',
                                     cv=kfold_cv,
                                     return_train_score=True)

nn_keras_gridsearchcv.fit(X_train, y_train, nn_model__callbacks=[es])

Epoch 1/100
45/45 - 0s - loss: 1.3405 - recall: 0.1601
Epoch 2/100
45/45 - 0s - loss: 1.1947 - recall: 0.2767
Epoch 3/100
45/45 - 0s - loss: 1.0901 - recall: 0.4494
Epoch 4/100
45/45 - 0s - loss: 1.0083 - recall: 0.6025
Epoch 5/100
45/45 - 0s - loss: 0.9429 - recall: 0.6896
Epoch 6/100
45/45 - 0s - loss: 0.8854 - recall: 0.7388
Epoch 7/100
45/45 - 0s - loss: 0.8386 - recall: 0.7683
Epoch 8/100
45/45 - 0s - loss: 0.7969 - recall: 0.7851
Epoch 9/100
45/45 - 0s - loss: 0.7582 - recall: 0.8090
Epoch 10/100
45/45 - 0s - loss: 0.7222 - recall: 0.8216
Epoch 11/100
45/45 - 0s - loss: 0.6882 - recall: 0.8301
Epoch 12/100
45/45 - 0s - loss: 0.6594 - recall: 0.8315
Epoch 13/100
45/45 - 0s - loss: 0.6362 - recall: 0.8371
Epoch 14/100
45/45 - 0s - loss: 0.6184 - recall: 0.8427
Epoch 15/100
45/45 - 0s - loss: 0.6028 - recall: 0.8511
Epoch 16/100
45/45 - 0s - loss: 0.5884 - recall: 0.8567
Epoch 17/100
45/45 - 0s - loss: 0.5756 - recall: 0.8610
Epoch 18/100
45/45 - 0s - loss: 0.5635 - recall: 0.8624
E

Epoch 37/100
45/45 - 0s - loss: 0.4539 - recall: 0.8764
Epoch 38/100
45/45 - 0s - loss: 0.4509 - recall: 0.8778
Epoch 39/100
45/45 - 0s - loss: 0.4484 - recall: 0.8750
Epoch 40/100
45/45 - 0s - loss: 0.4458 - recall: 0.8778
Epoch 41/100
45/45 - 0s - loss: 0.4433 - recall: 0.8834
Epoch 42/100
45/45 - 0s - loss: 0.4406 - recall: 0.8806
Epoch 43/100
45/45 - 0s - loss: 0.4387 - recall: 0.8764
Epoch 44/100
45/45 - 0s - loss: 0.4362 - recall: 0.8792
Epoch 45/100
45/45 - 0s - loss: 0.4340 - recall: 0.8806
Epoch 46/100
45/45 - 0s - loss: 0.4318 - recall: 0.8862
Epoch 47/100
45/45 - 0s - loss: 0.4298 - recall: 0.8890
Epoch 48/100
45/45 - 0s - loss: 0.4276 - recall: 0.8820
Epoch 49/100
45/45 - 0s - loss: 0.4254 - recall: 0.8919
Epoch 50/100
45/45 - 0s - loss: 0.4236 - recall: 0.8933
Epoch 51/100
45/45 - 0s - loss: 0.4218 - recall: 0.8961
Epoch 52/100
45/45 - 0s - loss: 0.4197 - recall: 0.8975
Epoch 53/100
45/45 - 0s - loss: 0.4178 - recall: 0.8947
Epoch 54/100
45/45 - 0s - loss: 0.4160 - recall:

Epoch 84/100
45/45 - 0s - loss: 0.3859 - recall: 0.9228
Epoch 85/100
45/45 - 0s - loss: 0.3828 - recall: 0.9228
Epoch 86/100
45/45 - 0s - loss: 0.3809 - recall: 0.9242
Epoch 87/100
45/45 - 0s - loss: 0.3779 - recall: 0.9199
Epoch 88/100
45/45 - 0s - loss: 0.3754 - recall: 0.9228
Epoch 89/100
45/45 - 0s - loss: 0.3732 - recall: 0.9213
Epoch 90/100
45/45 - 0s - loss: 0.3710 - recall: 0.9270
Epoch 91/100
45/45 - 0s - loss: 0.3691 - recall: 0.9228
Epoch 92/100
45/45 - 0s - loss: 0.3672 - recall: 0.9228
Epoch 93/100
45/45 - 0s - loss: 0.3653 - recall: 0.9298
Epoch 94/100
45/45 - 0s - loss: 0.3632 - recall: 0.9312
Epoch 95/100
45/45 - 0s - loss: 0.3616 - recall: 0.9326
Epoch 96/100
45/45 - 0s - loss: 0.3597 - recall: 0.9340
Epoch 97/100
45/45 - 0s - loss: 0.3582 - recall: 0.9284
Epoch 98/100
45/45 - 0s - loss: 0.3566 - recall: 0.9354
Epoch 99/100
45/45 - 0s - loss: 0.3548 - recall: 0.9340
Epoch 100/100
45/45 - 0s - loss: 0.3532 - recall: 0.9298
2/2 - 0s
26/26 - 0s
Epoch 1/100
45/45 - 0s - lo

Epoch 27/100
45/45 - 0s - loss: 0.6001 - recall: 0.7261
Epoch 28/100
45/45 - 0s - loss: 0.5903 - recall: 0.7331
Epoch 29/100
45/45 - 0s - loss: 0.5804 - recall: 0.7416
Epoch 30/100
45/45 - 0s - loss: 0.5710 - recall: 0.7486
Epoch 31/100
45/45 - 0s - loss: 0.5620 - recall: 0.7514
Epoch 32/100
45/45 - 0s - loss: 0.5541 - recall: 0.7640
Epoch 33/100
45/45 - 0s - loss: 0.5470 - recall: 0.7711
Epoch 34/100
45/45 - 0s - loss: 0.5409 - recall: 0.7767
Epoch 35/100
45/45 - 0s - loss: 0.5356 - recall: 0.7809
Epoch 36/100
45/45 - 0s - loss: 0.5305 - recall: 0.7837
Epoch 37/100
45/45 - 0s - loss: 0.5259 - recall: 0.7809
Epoch 38/100
45/45 - 0s - loss: 0.5215 - recall: 0.7851
Epoch 39/100
45/45 - 0s - loss: 0.5168 - recall: 0.7893
Epoch 40/100
45/45 - 0s - loss: 0.5126 - recall: 0.7907
Epoch 41/100
45/45 - 0s - loss: 0.5085 - recall: 0.7823
Epoch 42/100
45/45 - 0s - loss: 0.5050 - recall: 0.7851
Epoch 43/100
45/45 - 0s - loss: 0.5013 - recall: 0.7823
Epoch 44/100
45/45 - 0s - loss: 0.4980 - recall:

Epoch 74/100
45/45 - 0s - loss: 0.3865 - recall: 0.8287
Epoch 75/100
45/45 - 0s - loss: 0.3865 - recall: 0.8357
Epoch 76/100
45/45 - 0s - loss: 0.3847 - recall: 0.8343
Epoch 77/100
45/45 - 0s - loss: 0.3848 - recall: 0.8287
Epoch 78/100
45/45 - 0s - loss: 0.3839 - recall: 0.8385
Epoch 79/100
45/45 - 0s - loss: 0.3828 - recall: 0.8329
Epoch 80/100
45/45 - 0s - loss: 0.3824 - recall: 0.8343
Epoch 81/100
45/45 - 0s - loss: 0.3816 - recall: 0.8357
Epoch 82/100
45/45 - 0s - loss: 0.3809 - recall: 0.8385
Epoch 83/100
45/45 - 0s - loss: 0.3805 - recall: 0.8343
Epoch 84/100
45/45 - 0s - loss: 0.3800 - recall: 0.8385
Epoch 85/100
45/45 - 0s - loss: 0.3792 - recall: 0.8371
Epoch 86/100
45/45 - 0s - loss: 0.3784 - recall: 0.8371
Epoch 87/100
45/45 - 0s - loss: 0.3778 - recall: 0.8357
Epoch 88/100
45/45 - 0s - loss: 0.3775 - recall: 0.8441
Epoch 89/100
45/45 - 0s - loss: 0.3776 - recall: 0.8343
Epoch 90/100
45/45 - 0s - loss: 0.3761 - recall: 0.8343
Epoch 91/100
45/45 - 0s - loss: 0.3756 - recall:

...

Epoch 36/100
45/45 - 0s - loss: 0.5544 - recall: 0.5534
Epoch 37/100
45/45 - 0s - loss: 0.5708 - recall: 0.5520
Epoch 38/100
45/45 - 0s - loss: 0.5569 - recall: 0.5815
Epoch 39/100
45/45 - 0s - loss: 0.5753 - recall: 0.5449
2/2 - 0s
26/26 - 0s
Epoch 1/100
45/45 - 0s - loss: 0.6995 - recall: 0.3947
Epoch 2/100
45/45 - 0s - loss: 0.6604 - recall: 0.5253
Epoch 3/100
45/45 - 0s - loss: 0.6561 - recall: 0.6433
Epoch 4/100
45/45 - 0s - loss: 0.6468 - recall: 0.7149
Epoch 5/100
45/45 - 0s - loss: 0.6319 - recall: 0.7261
Epoch 6/100
45/45 - 0s - loss: 0.6310 - recall: 0.7542
Epoch 7/100
45/45 - 0s - loss: 0.6182 - recall: 0.7697
Epoch 8/100
45/45 - 0s - loss: 0.6094 - recall: 0.8006
Epoch 9/100
45/45 - 0s - loss: 0.6027 - recall: 0.8174
Epoch 10/100
45/45 - 0s - loss: 0.6044 - recall: 0.8132
Epoch 11/100
45/45 - 0s - loss: 0.5855 - recall: 0.8258
Epoch 12/100
45/45 - 0s - loss: 0.5593 - recall: 0.8357
Epoch 13/100
45/45 - 0s - loss: 0.5645 - recall: 0.8357
Epoch 14/100
45/45 - 0s - loss: 0.538

Epoch 46/100
45/45 - 0s - loss: 0.4565 - recall: 0.7177
Epoch 47/100
45/45 - 0s - loss: 0.4489 - recall: 0.7149
2/2 - 0s
26/26 - 0s
Epoch 1/100
45/45 - 0s - loss: 0.8488 - recall: 0.5169
Epoch 2/100
45/45 - 0s - loss: 0.7729 - recall: 0.4930
Epoch 3/100
45/45 - 0s - loss: 0.7092 - recall: 0.4761
Epoch 4/100
45/45 - 0s - loss: 0.7088 - recall: 0.4242
Epoch 5/100
45/45 - 0s - loss: 0.6974 - recall: 0.4256
Epoch 6/100
45/45 - 0s - loss: 0.6859 - recall: 0.4354
Epoch 7/100
45/45 - 0s - loss: 0.6948 - recall: 0.3862
Epoch 8/100
45/45 - 0s - loss: 0.6766 - recall: 0.3975
Epoch 9/100
45/45 - 0s - loss: 0.6656 - recall: 0.4143
Epoch 10/100
45/45 - 0s - loss: 0.6460 - recall: 0.4691
Epoch 11/100
45/45 - 0s - loss: 0.6521 - recall: 0.4522
Epoch 12/100
45/45 - 0s - loss: 0.6348 - recall: 0.4565
Epoch 13/100
45/45 - 0s - loss: 0.6352 - recall: 0.4803
Epoch 14/100
45/45 - 0s - loss: 0.6125 - recall: 0.5098
Epoch 15/100
45/45 - 0s - loss: 0.6136 - recall: 0.5169
Epoch 16/100
45/45 - 0s - loss: 0.621

Epoch 40/100
45/45 - 0s - loss: 0.4907 - recall: 0.5871
Epoch 41/100
45/45 - 0s - loss: 0.4971 - recall: 0.5857
Epoch 42/100
45/45 - 0s - loss: 0.5081 - recall: 0.5435
Epoch 43/100
45/45 - 0s - loss: 0.4952 - recall: 0.5576
2/2 - 0s
26/26 - 0s
Epoch 1/100
45/45 - 0s - loss: 0.7410 - recall: 0.3567
Epoch 2/100
45/45 - 0s - loss: 0.6915 - recall: 0.4888
Epoch 3/100
45/45 - 0s - loss: 0.6646 - recall: 0.5787
Epoch 4/100
45/45 - 0s - loss: 0.6579 - recall: 0.6489
Epoch 5/100
45/45 - 0s - loss: 0.6310 - recall: 0.7317
Epoch 6/100
45/45 - 0s - loss: 0.6270 - recall: 0.7669
Epoch 7/100
45/45 - 0s - loss: 0.6028 - recall: 0.8076
Epoch 8/100
45/45 - 0s - loss: 0.5941 - recall: 0.8146
Epoch 9/100
45/45 - 0s - loss: 0.5753 - recall: 0.8174
Epoch 10/100
45/45 - 0s - loss: 0.5740 - recall: 0.8160
Epoch 11/100
45/45 - 0s - loss: 0.5776 - recall: 0.8020
Epoch 12/100
45/45 - 0s - loss: 0.5565 - recall: 0.8539
Epoch 13/100
45/45 - 0s - loss: 0.5583 - recall: 0.8708
Epoch 14/100
45/45 - 0s - loss: 0.540

Epoch 10/100
45/45 - 0s - loss: 0.6395 - recall: 0.4292
Epoch 11/100
45/45 - 0s - loss: 0.6188 - recall: 0.4600
Epoch 12/100
45/45 - 0s - loss: 0.6348 - recall: 0.4151
Epoch 13/100
45/45 - 0s - loss: 0.6003 - recall: 0.4769
Epoch 14/100
45/45 - 0s - loss: 0.6056 - recall: 0.4684
Epoch 15/100
45/45 - 0s - loss: 0.6052 - recall: 0.4558
Epoch 16/100
45/45 - 0s - loss: 0.5942 - recall: 0.5133
Epoch 17/100
45/45 - 0s - loss: 0.5832 - recall: 0.5077
Epoch 18/100
45/45 - 0s - loss: 0.5790 - recall: 0.5666
Epoch 19/100
45/45 - 0s - loss: 0.5547 - recall: 0.5905
Epoch 20/100
45/45 - 0s - loss: 0.5607 - recall: 0.5989
Epoch 21/100
45/45 - 0s - loss: 0.5645 - recall: 0.5806
Epoch 22/100
45/45 - 0s - loss: 0.5677 - recall: 0.5750
2/2 - 0s
26/26 - 0s
Epoch 1/100
45/45 - 0s - loss: 0.8805 - recall: 0.5997
Epoch 2/100
45/45 - 0s - loss: 0.7999 - recall: 0.5463
Epoch 3/100
45/45 - 0s - loss: 0.7462 - recall: 0.5056
Epoch 4/100
45/45 - 0s - loss: 0.7138 - recall: 0.5463
Epoch 5/100
45/45 - 0s - loss: 0

GridSearchCV(cv=StratifiedKFold(n_splits=15, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('knn_impute', KNNImputer()),
                                       ('smote_sample', SMOTE(random_state=42)),
                                       ('std_scale', StandardScaler()),
                                       ('nn_model',
                                        <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x0000021386BA1430>)]),
             param_grid={'nn_model__dropout_rate': [0.0, 0.2, 0.4],
                         'nn_model__layer_1': [5, 6],
                         'nn_model__layer_2': [3, 4],
                         'nn_model__regularisation_alpha': [0.1, 0.01, 0.001,
                                                            0.0001, 1e-05]},
             return_train_score=True, scoring='recall')

<br>  

## <span style='color:blue'>Section 7: Save results</span>  

In [7]:
temp_df_dt = []
for j in range(len(nn_keras_gridsearchcv.cv_results_['params'])):
    for i in range(kfold_cv.n_splits):
        over_fit = (nn_keras_gridsearchcv.cv_results_['split'+str(i)+'_test_score'][j]-nn_keras_gridsearchcv.cv_results_['split'+str(i)+'_train_score'][j])/nn_keras_gridsearchcv.cv_results_['split'+str(i)+'_train_score'][j]*100
        temp_df_dt.append([nn_keras_gridsearchcv.cv_results_['params'][j],
                           nn_keras_gridsearchcv.cv_results_['params'][j]['nn_model__dropout_rate'],
                           nn_keras_gridsearchcv.cv_results_['params'][j]['nn_model__layer_1'],
                           nn_keras_gridsearchcv.cv_results_['params'][j]['nn_model__layer_2'],
                           nn_keras_gridsearchcv.cv_results_['params'][j]['nn_model__regularisation_alpha'],
                           nn_keras_gridsearchcv.cv_results_['split'+str(i)+'_train_score'][j],
                           nn_keras_gridsearchcv.cv_results_['split'+str(i)+'_test_score'][j],
                           over_fit])
temp_df_dt = pd.DataFrame(temp_df_dt, columns=['parameters', 'dropout', 'layer_1', 'layer_2',
                                               'alpha', 'training', 'validation', 'overfit_%'])
print(temp_df_dt)
temp_df_dt.to_csv('../data/code_6a_nn_train_validate_recall.csv', na_rep='NaN', index_label='index')

                                            parameters  dropout  layer_1  \
0    {'nn_model__dropout_rate': 0.0, 'nn_model__lay...      0.0        5   
1    {'nn_model__dropout_rate': 0.0, 'nn_model__lay...      0.0        5   
2    {'nn_model__dropout_rate': 0.0, 'nn_model__lay...      0.0        5   
3    {'nn_model__dropout_rate': 0.0, 'nn_model__lay...      0.0        5   
4    {'nn_model__dropout_rate': 0.0, 'nn_model__lay...      0.0        5   
..                                                 ...      ...      ...   
895  {'nn_model__dropout_rate': 0.4, 'nn_model__lay...      0.4        6   
896  {'nn_model__dropout_rate': 0.4, 'nn_model__lay...      0.4        6   
897  {'nn_model__dropout_rate': 0.4, 'nn_model__lay...      0.4        6   
898  {'nn_model__dropout_rate': 0.4, 'nn_model__lay...      0.4        6   
899  {'nn_model__dropout_rate': 0.4, 'nn_model__lay...      0.4        6   

     layer_2    alpha  training  validation  overfit_%  
0          3  0.10000  0.82882

3 Jan 2022